In [1]:
# импортируем библиотеки

import pandas as pd
import numpy as np

In [ ]:
# загружаем данные из excel

data_1 = pd.read_excel(r'C:\Users\OneDrive\Рабочий стол\Задание\Список ИНН.xlsx')
data_2 = pd.read_excel(r'C:\Users\OneDrive\Рабочий стол\Задание\Список КМ.xlsx')

In [3]:
# посмотрим как загрузились данные

for i in [data_1, data_2]:
    print(i.head())
    print('----------------------------')
    print(i.info())
    print('----------------------------')

        ИНН    Холдинг Сотрудничество Головное отделение  Потенциальный доход
0  14361977        NaN         Клиент                ГО1              5702680
1   1454444        NaN         Клиент                ГО2              4129018
2  14689141  Холдинг_1      Не клиент                ГО1              9645470
3  14709947        NaN      Не клиент                ГО2              4375217
4  14767489        NaN      Не клиент                ГО3              7749114
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ИНН                  1000 non-null   int64 
 1   Холдинг              382 non-null    object
 2   Сотрудничество       1000 non-null   object
 3   Головное отделение   1000 non-null   object
 4   Потенциальный доход  1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB
None

In [4]:
# по условию "если ИНН не входит в холдинг, то считается уникальным ID"
# заменим все NaN в столбце холдинг на "Уникальный_ID"

data_1['Холдинг'] = data_1['Холдинг'].fillna("Уникальный_ID")

In [5]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ИНН                  1000 non-null   int64 
 1   Холдинг              1000 non-null   object
 2   Сотрудничество       1000 non-null   object
 3   Головное отделение   1000 non-null   object
 4   Потенциальный доход  1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [6]:
data_1.head()

,ИНН,Холдинг,Сотрудничество,Головное отделение,Потенциальный доход
0,14361977,Уникальный_ID,Клиент,ГО1,5702680
1,1454444,Уникальный_ID,Клиент,ГО2,4129018
2,14689141,Холдинг_1,Не клиент,ГО1,9645470
3,14709947,Уникальный_ID,Не клиент,ГО2,4375217
4,14767489,Уникальный_ID,Не клиент,ГО3,7749114


In [7]:
data_2.head()

,ФИО,Головное отделение
0,Пушкин Александр Сергеевич,ГО1
1,Гоголь Николай Васильевич,ГО3
2,Достоевский Федор Михайлович,ГО2
3,Толстой Лев Николаевич,ГО1
4,Чехов Антон Павлович,ГО3


В итоге у нас есть чистый датасет. Пропусков нет. Типы полей корректные. Можно приступать к анализу.

Необходимо сформировать пакет для каждого КМ с учетом следующих критериев:

*	Кол-во уникальных ID в пакете не более 150.
*	Доля не клиентов в пакете не более 35%
*	Пакет должен содержать ИНН только из данного головного отделения где присутствует КМ.
*	Пакет должен быть сбалансированным по потенциалу у каждого КМ.

Проверим для начала выполнимы ли условия.

In [8]:
# найдем сумму потенциального дохода по отделениям

data_1.groupby('Головное отделение')['Потенциальный доход'].sum()

Головное отделение
ГО1    1502093205
ГО2    1708283991
ГО3    1675455030
Name: Потенциальный доход, dtype: int64

In [9]:
# найдем общее число отделений в датасете

data_1['Головное отделение'].value_counts()

Головное отделение
ГО2    344
ГО3    341
ГО1    315
Name: count, dtype: int64

In [10]:
# посмотрим на распределение уникальных ID по отделениям

data_1.query("Холдинг == 'Уникальный_ID'").groupby('Головное отделение')['Холдинг'].count()

Головное отделение
ГО1    173
ГО2    302
ГО3    143
Name: Холдинг, dtype: int64

In [11]:
# найдем долю не клиентов по отделениям

data_1[data_1['Сотрудничество'] == 'Не клиент'].\
    groupby('Головное отделение')['Сотрудничество'].count() / data_1.\
    groupby('Головное отделение')['Сотрудничество'].count()

Головное отделение
ГО1    0.526984
ГО2    0.526163
ГО3    0.480938
Name: Сотрудничество, dtype: float64

Итак. У нас есть 5-ть менеджеров. Два работают в ГО1, один работает в ГО2 и ещё два работают в ГО3.
Т.о.

* "Количество уникальных ID в пакете не более 150"  -  это скорее относится к сотруднику в ГО2. Т.к. в других отделениях, при равномерном делении, их число итак не превысит отметку в 150.

* Доля не клиентов в пакете не более 35%  -  это уже можно отнести ко всем ГО. Но условие невыполнимо при равномерном распределении для всех. Т.к. общая доля превышает допустимое значение. Единственный вариант соблюсти условие сократить число ИНН в пакете т.о., чтобы было больше клиентов, чем не клиентов.

* Пакет должен быть только из ИНН, где работает агент   -  это условие выполнимо. Данных достаточно.

* Пакет должен быть сбалансированным по потенциальному доходу для каждого  -  Здесь больше всего выделяется ГО2, т.к. суммарное значение больше всех, а работает всего один КМ. И при равномерном распределении всех точек выполнить данное условие невозможно.


Учитывая выше изложенное, а также то, что нам не был задан объём пакета, который должен быть у каждого сотрудника. Можно предложить следующий вариант решения задачи:

1. Разбить датасет на несколько по значению головного отделения

2. Отсортировать датасеты по "клиент/не клиент" и по убыванию потенциального дохода

3. Убрать строки из датасетов с "дешевыми не клиентами" и тем самым уменьшить их наличие до необходимого уровня в 0.35

4. Провести разбивку датасетов на "пакеты" и проверить их соответствие параметрам задачи

5. Собрать "пакеты" в единый датафрейм, провести сравнительный анализ

Поскольку у нас нет условий по величине (количеству ИНН) пакетов и точно не определено понятие "сбалансированного пакета", мы обусловим, что количество ИНН в пакете может быть каким угодно. 

А сбалансированными пакеты будут считаться, если величина отношения пакета с минимальным значением и пакета с максимальным значением будет менее 1%.

In [12]:
# сформируем по каждому ГО свой датасет. И отсортируем по клиент/не клиент и потенциальному доходу

df1 = data_1[data_1['Головное отделение'] == 'ГО1'].sort_values(by=['Сотрудничество', 'Потенциальный доход'], ascending=[True, False])
df2 = data_1[data_1['Головное отделение'] == 'ГО2'].sort_values(by=['Сотрудничество', 'Потенциальный доход'], ascending=[True, False])
df3 = data_1[data_1['Головное отделение'] == 'ГО3'].sort_values(by=['Сотрудничество', 'Потенциальный доход'], ascending=[True, False])

In [13]:
# создадим функцию для проверки параметров задачи

def chek_df (data):
    print('Доля не клиентов:', data[data['Сотрудничество'] == 'Не клиент'].\
    groupby('Головное отделение')['Сотрудничество'].count() / data.\
    groupby('Головное отделение')['Сотрудничество'].count())
    print('--------------------------------------')
    print('Количество уникальных id:', data.query("Холдинг == 'Уникальный_ID'").groupby('Головное отделение')['Холдинг'].count())
    print('--------------------------------------')
    print('Суммарный потенциальный доход:', data['Потенциальный доход'].sum())
    print('--------------------------------------')

In [14]:
# удалим строчки из числа не клиентов с минимальным доходом

df1 = df1.iloc[:-86]
df2 = df2.iloc[:-94]
df3 = df3.iloc[:-70]

In [15]:
# проверим что доля не клиентов не превышает 0.35

for i in [df1, df2, df3]:
    chek_df(i)

Доля не клиентов: Головное отделение
ГО1    0.349345
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО1    125
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 1298717462
--------------------------------------
Доля не клиентов: Головное отделение
ГО2    0.348
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО2    222
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 1443784863
--------------------------------------
Доля не клиентов: Головное отделение
ГО3    0.346863
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО3    114
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 1514447936
--------------------------------------


In [16]:
# создадим счетчики и переменные с лимитами. 
# т.к. в ГО1 меньше всего записей и сумма потенциального дохода не превышает 1_300_000_000
# тогда установим лимит по уровню дохода на одного сотрудника в размере 650_000_000

uniq_id = 150
level_income = 650_000_000
emp_go1_1 = []
emp_go1_2 = []
emp_go2 = []
emp_go3_1 = []
emp_go3_2 = []
sum_income_emp_go1_1 = 0
sum_income_emp_go1_2 = 0
sum_income_emp_go2 = 0
sum_income_emp_go3_1 = 0
sum_income_emp_go3_2 = 0
count_uniq_id_emp_go1_1 = 0
count_uniq_id_emp_go1_2 = 0
count_uniq_id_emp_go2 = 0
count_uniq_id_emp_go3_1 = 0
count_uniq_id_emp_go3_2 = 0

In [17]:
# распределим ИНН между сотрудниками в ГО1

for index, record in df1.iterrows():
    if count_uniq_id_emp_go1_1 < uniq_id and sum_income_emp_go1_1 < sum_income_emp_go1_2:
        emp_go1_1.append(record)
        count_uniq_id_emp_go1_1 += 1 if record['Холдинг'] == 'Уникальный_ID' else 0
        sum_income_emp_go1_1 += record['Потенциальный доход']
    elif count_uniq_id_emp_go1_2 < uniq_id:
        emp_go1_2.append(record)
        count_uniq_id_emp_go1_2 += 1 if record['Холдинг'] == 'Уникальный_ID' else 0
        sum_income_emp_go1_2 += record['Потенциальный доход']
    else:
        continue

In [18]:
# распределим ИНН между сотрудниками в ГО3

for index, record in df3.iterrows():
    if count_uniq_id_emp_go3_1 < uniq_id and sum_income_emp_go3_1 < sum_income_emp_go3_2 and sum_income_emp_go3_1 <= level_income:
        emp_go3_1.append(record)
        count_uniq_id_emp_go3_1 += 1 if record['Холдинг'] == 'Уникальный_ID' else 0
        sum_income_emp_go3_1 += record['Потенциальный доход']
    elif count_uniq_id_emp_go3_2 < uniq_id and sum_income_emp_go3_2 <= level_income:
        emp_go3_2.append(record)
        count_uniq_id_emp_go3_2 += 1 if record['Холдинг'] == 'Уникальный_ID' else 0
        sum_income_emp_go3_2 += record['Потенциальный доход']
    else:
        continue

In [19]:
# определим ИНН для сотрудника в ГО2

for index, record in df2.iterrows():
    if count_uniq_id_emp_go2 < uniq_id and sum_income_emp_go2 <= level_income:
        emp_go2.append(record)
        count_uniq_id_emp_go2 += 1 if record['Холдинг'] == 'Уникальный_ID' else 0
        sum_income_emp_go2 += record['Потенциальный доход']
    else:
        continue

In [20]:
# соберем пакеты в датафреймы

df_emp_go1_1 = pd.DataFrame(emp_go1_1)
df_emp_go1_2 = pd.DataFrame(emp_go1_2)
df_emp_go2 = pd.DataFrame(emp_go2)
df_emp_go3_1 = pd.DataFrame(emp_go3_1)
df_emp_go3_2 = pd.DataFrame(emp_go3_2)

In [21]:
# проверим что разбиение прошло успешно

for i in [df_emp_go1_1, df_emp_go1_2, df_emp_go2, df_emp_go3_1, df_emp_go3_2]:
    chek_df(i)

Доля не клиентов: Головное отделение
ГО1    0.347826
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО1    61
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 649350976
--------------------------------------
Доля не клиентов: Головное отделение
ГО1    0.350877
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО1    64
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 649366486
--------------------------------------
Доля не клиентов: Головное отделение
ГО2   NaN
Name: Сотрудничество, dtype: float64
--------------------------------------
Количество уникальных id: Головное отделение
ГО2    87
Name: Холдинг, dtype: int64
--------------------------------------
Суммарный потенциальный доход: 652360260
--------------------------------------
Доля не клиен

In [22]:
data_2

,ФИО,Головное отделение
0,Пушкин Александр Сергеевич,ГО1
1,Гоголь Николай Васильевич,ГО3
2,Достоевский Федор Михайлович,ГО2
3,Толстой Лев Николаевич,ГО1
4,Чехов Антон Павлович,ГО3


In [23]:
# распределим пакеты между сотрудниками

df_emp_go1_1['ФИО'] = 'Пушкин Александр Сергеевич'
df_emp_go1_2['ФИО'] = 'Толстой Лев Николаевич'
df_emp_go2['ФИО'] = 'Достоевский Федор Михайлович'
df_emp_go3_1['ФИО'] = 'Гоголь Николай Васильевич'
df_emp_go3_2['ФИО'] = 'Чехов Антон Павлович'

In [24]:
# объединим в один датасет

data_total = pd.concat([df_emp_go1_1, df_emp_go1_2, df_emp_go2, df_emp_go3_1, df_emp_go3_2])
print(data_total.info())

<class 'pandas.core.frame.DataFrame'>
Index: 561 entries, 721 to 481
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ИНН                  561 non-null    int64 
 1   Холдинг              561 non-null    object
 2   Сотрудничество       561 non-null    object
 3   Головное отделение   561 non-null    object
 4   Потенциальный доход  561 non-null    int64 
 5   ФИО                  561 non-null    object
dtypes: int64(2), object(4)
memory usage: 30.7+ KB
None


In [25]:
# найдем число уникальных id, которых в пакете у каждого должно быть меньше 150

data_total.query("Холдинг == 'Уникальный_ID'").groupby('ФИО')['Холдинг'].count()

ФИО
Гоголь Николай Васильевич       54
Достоевский Федор Михайлович    87
Пушкин Александр Сергеевич      61
Толстой Лев Николаевич          64
Чехов Антон Павлович            47
Name: Холдинг, dtype: int64

In [26]:
# найдем долю не клиентов по сотрудникам

round(data_total[data_total['Сотрудничество'] == 'Не клиент'].\
    groupby('ФИО')['Сотрудничество'].count() / data_total.\
    groupby('ФИО')['Сотрудничество'].count(), 2)

ФИО
Гоголь Николай Васильевич       0.25
Достоевский Федор Михайлович     NaN
Пушкин Александр Сергеевич      0.35
Толстой Лев Николаевич          0.35
Чехов Антон Павлович            0.25
Name: Сотрудничество, dtype: float64

In [27]:
# найдем суммарный потенциальный доход для каждого КМ

data_total.groupby('ФИО')['Потенциальный доход'].sum()

ФИО
Гоголь Николай Васильевич       653918171
Достоевский Федор Михайлович    652360260
Пушкин Александр Сергеевич      649350976
Толстой Лев Николаевич          649366486
Чехов Антон Павлович            653971410
Name: Потенциальный доход, dtype: int64

In [ ]:
# посчитаем отношение самого min/max доходов по сотрудникам

print('Величина отношения min/max потенциальных доходов равна:', round(649350976/653971410,3) )

Величина отношения min/max потенциальных доходов равна: 0.993


ВЫВОД:

Нам удалось сформировать пакеты для каждого КМ т.о. чтобы были соблюдены все условия:

1. Кол-во уникальных ID в пакете не более 150.
2. Доля не клиентов в пакете не более 35%
3. Пакет содержит ИНН только из данного головного отделения где присутствует КМ.
4. Пакет должен быть сбалансированным по потенциалу у каждого КМ.

Расхождение в потенциальном доходе у КМ с наименьшим значением и КМ с наибольшим значением составило менее 1% (если точнее, то 0.7%)
Т.о. можем считать, что пакеты сбалансированы.